In [1]:
import pandas as pd
import numpy as pd
from psaw import PushshiftAPI
import psycopg2 as pg
import psycopg2.extras
import datetime

In [2]:
start_time = int(datetime.datetime(2022,5,14).timestamp())

In [3]:
api = PushshiftAPI()

In [4]:
### Create a generator onject with {limit} submissions from {subreddit}
### Showing only fields in {filter}
submissions = list(api.search_submissions(after= start_time,
                            subreddit = 'wallstreetbets',
                            filter = ["url", "author", "title", "subreddit"],
                            limit = 1000
                                         ))

for submission in submissions:
#     print(submission.created_utc)
#     print(submission.title)
#     print(submission.url)
    
    words = submission.title.split()
    cashtags = list(set(filter(lambda word: word.lower().startswith("$"), words)))
    
#     if len(cashtags) > 0:
#         print(cashtags)
    

['$spy']
['$700/share']
['$500,']
['$38K']
['$AMC.']
['$175', '$FB']
['$SNAP']
['$1.5k']
['$69']
['$3k']
['$1.35.', '$60']
['$2.5K']
['$2.5K']
['$GME']
['$AMC']
['$90k']
['$90k']
['$AMC']
['$bll']
['$90k']
['$AMC']
['$40']
['$500']
['$ZM']
['$500']
['$QQQ']
['$PRPL']
['$TROJAN']
['$TROJAN']
['$ZIM-']
['$BBAI:']
['$1530']
['$TSLA']
['$BYND']
['$RDBX']
['$BBAI,']
['$1000']
['$100']
['$1']
['$BBAI']
['$1']
['$SU']
['$BIDEN']
['$WSG']
['$BIDEN']
['$BIDEN']
['$BIDEN']
['$SIGA:']
['$SPY']
['$SPY']
['$30k', '$36k.']
['$GOVX']
['$SIGA', '$GOVX']
['$2,000']
['$200,000', '$1']
['$1']
['$250k']
['$LUNA']
['$200,000', '$1']
['$40k']
['$AMC']
['$420.69']
['$RDBX']
['$100k']
['$SRS?']
['$100k']
['$100k']
['$10']
['$SPY']
['$SPY']
['$SPX']
['$BAVA', '$EBS']
['$BVNRY', '$EBS']
['$CNSP']
['$CNSP']
['$APPL']
['$414']
['$300', '$94']
['$Luna', '$Lunc?']
['$SPY']
['$RRGB,']
['$GME', '$AMC']
['$BBBY']
['$BBBY']
['$10B']
['$BBBY']
['$BBBY']
['$DUST']
['$6k', '$TSLA', '$46k']


In [6]:
connection = pg.connect(
   database="postgres", user='postgres', password='admin', host='localhost', port= '5432'
)

In [7]:
cursor = connection.cursor()

In [8]:
cursor.execute("""CREATE EXTENSION IF NOT EXISTS timescaledb CASCADE;""")

In [9]:
cursor.execute('SELECT * from pg_exteansion;')
results = cursor.fetchall()
for result in results:
    print(result)


In [10]:
results = cursor.fetchall()
for result in results:
    print(result)


(13740, 'plpgsql', 10, 11, False, '1.0', None, None)
(16384, 'adminpack', 10, 11, False, '2.1', None, None)
(29444, 'timescaledb', 10, 2200, False, '2.6.1', [29461, 29462, 29483, 29496, 29510, 29509, 29528, 29527, 29543, 29542, 29565, 29581, 29582, 29598, 29610, 29611, 29653, 29660, 29681, 29693, 29703, 29707, 29723, 29742, 29757, 29778, 29784, 29781], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'WHERE id >= 1000', "\r\n  WHERE KEY = 'exported_uuid' ", '', '', '', '', '', '', '', '', '', '', ''])


In [11]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS  mention (
    stock_id VARCHAR(255),
    dt TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    message TEXT NOT NULL,
    source TEXT NOt NULL, --reddit (subreddit), twitter ...
    url TEXT NOT NULL,
    PRIMARY KEY (stock_id, dt),
    CONSTRAINT fk_mention_stock FOREIGN KEY (stock_id) REFERENCES tickers (internal_id)
    );
    SELECT create_hypertable('Mention', 'dt');
""")

In [12]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [13]:
cursor.execute("""
SELECT * FROM tickers
""")
rows = cursor.fetchall()
stocks = {}
for row in rows:
    stocks[f"${row['symbol']}"] = row["internal_id"]
    
    
#### insert of mention table (place inside loop)
for cashtag in cashtags:
    submitted_time = datetime.datetime.fromtimestamp(submission.created_utc).isoformat()
    
    try:
        cursor.execute("""
            INSERT INTO mention (dt, stock, message, source, url)
            VALUES (%s, %s, %s, 'wallstreetbets', %s)
        """,(submitted_time, stocks[cashtag], submission.title, submission.url))
        connection.commit()
    except Expection as e:
        print(e)
        conection.rollback()

In [14]:
cursor.execute("""
SELECT * FROM mention
""")
rows = cursor.fetchall()
for i in rows:
    print(i)

In [15]:
cursor.close()
connection.close()